<a href="https://colab.research.google.com/github/Nikhila-KS/Unravel_ML/blob/main/1.)data_preprocessing_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preprocessing Tools

## Importing the libraries

In [57]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# NumPy:
# NumPy is a Python library that stands for "Numerical Python." It provides a powerful set of tools and functions for working 
# with arrays and matrices, along with a collection of mathematical functions to perform computations efficiently. NumPy is 
# widely used in scientific and numerical computing applications.

# Matplotlib:
# Matplotlib is a plotting library for Python. It provides a flexible and comprehensive set of functions for creating various 
# types of static, animated, and interactive visualizations. Matplotlib is widely used for creating plots, charts, histograms, 
# and other graphical representations of data.

# Pandas:
# Pandas is a Python library that provides data manipulation and analysis tools, primarily through its DataFrame data structure.
# It offers efficient data handling capabilities, such as reading and writing data from various file formats, data alignment, 
# data cleaning, reshaping, merging, and performing statistical operations on data. Pandas is widely used in data analysis and
# data science workflows.

# NumPy is focused on numerical computations and array manipulation, Matplotlib is used for creating visualizations, 
# and Pandas is primarily used for data manipulation and analysis. 

## Importing the dataset

In [58]:
dataset = pd.read_csv('Data.csv')
x= dataset.iloc[ : ,:-1].values # feature matrix
y= dataset.iloc[ : , -1].values # dependend variable vector

In [59]:
print(x)

[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 nan]
 ['France' 35.0 58000.0]
 ['Spain' nan 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]


In [60]:
print(y)

['No' 'Yes' 'No' 'No' 'Yes' 'Yes' 'No' 'Yes' 'No' 'Yes']


## Taking care of missing data

In [61]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan , strategy='mean') # nana means null/none
imputer.fit(x[ : ,1:3])        # fit and transform are methods defined in the SimpleImputer class
x[ : ,1:3]=imputer.transform(x[ : ,1:3])

In [62]:
print(x)

[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 63777.77777777778]
 ['France' 35.0 58000.0]
 ['Spain' 38.77777777777778 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]


## Encoding Categorical data


### Encoding the independent variables

In [63]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
# create obj of ColumnTransformer class

ct= ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[0])],remainder='passthrough')
# In the first argument we actually have to specify three things., First, the kind of transformation which is encoding. 
# Second, what kind of encoding we want to dowhich is, one hot encoding
# And third, the indexes of the columns we want to encode

# In the second argument remainder.Here we want to specify in quote the following code name, which is passthrough, and which is a code name
# that will say that we indeed want to keep, the columns that won't be applied some transformation, that won't be one hot encoded
# Which are of course age and salary.

x= np.array(ct.fit_transform(x))
#column transformer class actually has a method called fit transform which will do exactly the process of fitting  and transforming at once at the same time.
# the fit transfer method actually doesn't return the output as a NumPy array.So here we want to force the output of this fit transfer method to be a NumPy array.

print(x)

[[1.0 0.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [0.0 1.0 0.0 30.0 54000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 1.0 0.0 40.0 63777.77777777778]
 [1.0 0.0 0.0 35.0 58000.0]
 [0.0 0.0 1.0 38.77777777777778 52000.0]
 [1.0 0.0 0.0 48.0 79000.0]
 [0.0 1.0 0.0 50.0 83000.0]
 [1.0 0.0 0.0 37.0 67000.0]]


### Encoding the dependent variable


In [64]:
# for the dependent variable because indeed it has a text format.No. And yes, and we would just like to convert these strings 
# into zero and one respectively. And to do this, well that's very simple we're gonna use another class called "Label Encoder"

from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()
y= le.fit_transform(y) 
print(y)

[0 1 0 0 1 1 0 1 0 1]


## Splitting the dataset into Training set and Test set

**Let me explain the 'why'we have to apply features scaling after splitting the data set into the training set and a test set.** 

It's really obvious.
It is for this simple reason that the test set is supposed to be a brand new set on which you're going to evaluate your machine learning model. So it's exactly like, you know, your training your machine learning model on your training set,and then later on, you know, after it is trained you're going to deploy it on new observations. So what this means is that the test set is something you're not supposed to work with for the training. And feature scaling is, as you will see, a technique that will get the mean and the standard deviation of your feature, you know, in order to perform the scaling. So, if we apply feature scaling before the split then it will actually get the mean and the standard deviation of all the values,including the ones in the test set. And since the test set is something you're not supposed to have, you know, like some future data in production, well you know, applying feature scaling on the original data set, before the split, would cause some what we call information leakage on the test set.You know, we would grab some information from the test set, which we're not supposed to get, because it is supposed to be new data with new observations.
So remember this, the essential reason why you should not apply feature scaling before the split is to prevent information leakage on the test set; which you're not supposed to have until the training is done.

In [65]:
# splitting the data set into the training set and a test set consists of making two separate sets.
# One training set where you're gonna train your machinery model on existing observations,
# and one test set where you're going to evaluate the performance of your model on new observations.

# feature scaling simply consists of scaling all your variables, all your features, actually,
# to make sure they all take values in the same scale. And we do this so as to prevent one feature
# to dominate the other; which therefore would be neglected by the machine learning model.

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=1)


In [66]:
print('X TEST - \n',x_test)

X TEST - 
 [[0.0 1.0 0.0 30.0 54000.0]
 [1.0 0.0 0.0 37.0 67000.0]]


In [67]:
print('X Train - \n',x_train)

X Train - 
 [[0.0 0.0 1.0 38.77777777777778 52000.0]
 [0.0 1.0 0.0 40.0 63777.77777777778]
 [1.0 0.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [1.0 0.0 0.0 48.0 79000.0]
 [0.0 1.0 0.0 50.0 83000.0]
 [1.0 0.0 0.0 35.0 58000.0]]


In [68]:

print('Y TEST - \n',y_test)

Y TEST - 
 [0 1]


In [69]:
print('Y Train - \n',y_train)

Y Train - 
 [0 1 0 0 1 1 0 1]


**Data Splitting train_test_split()**
A function by scikit-learn, which contains a function called train_test_split.

And this function is to create four separate sets,actually not two, but four, because we will actually create a pair of matrix of features dependent variable for the training set, and another pair of matrix of features dependent variable for the test set.
So we're going to get four sets, X train, which is the matrix of features of the training set. X test, which is the matrix of features of the test set Y train, which is the dependent variable of the training set, and Y test, which is the dependent variable of the test set. 

For the training, it'll expect X train and Y train as input in a method actually called the fit method. And for the predictions, also called inference, these models will predict X test.

There are some parameters. It expects, well, the combination of the matrix of features X, and the dependent variable vector Y And that's the first two inputs of this function. We still have to input two more arguments which are going to be first the split size, because we're not going to split this data set
into a training set and a test set of the same size.
Well, I recommend to have 80% observation in the training set, and 20% in the test set.They will be taken randomly, but eight of them will go into the training set and two in the test set if we have 10 items.Now we'll add one final argument just for teaching purposes so that we can have the same results displayed in here,Well, to make sure we have the same random factors, we'll just set here, random_state = 1, right? We're just fixing the seed here so that we'll get the same split and therefore the same training set, and same test set.

## Feature Scaling

In [70]:
# feature scaling which will allow to put all our features on the same scale.
# Why do we need to do this?
# Well, that's because for some of the machinery models that's in order to avoid some features to be dominated by other 
# features in such a way that the dominated features are not even considered by the machinery model.

# you also need to be aware that we won't have to apply feature scaling for all the machinery models, just for some of them.

from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
x_train[:,3:]= sc.fit_transform(x_train[:,3:])
x_test[:,3:]=sc.transform(x_test[:,3:])


In [71]:
print(x_train)

[[0.0 0.0 1.0 -0.19159184384578545 -1.0781259408412425]
 [0.0 1.0 0.0 -0.014117293757057777 -0.07013167641635372]
 [1.0 0.0 0.0 0.566708506533324 0.633562432710455]
 [0.0 0.0 1.0 -0.30453019390224867 -0.30786617274297867]
 [0.0 0.0 1.0 -1.9018011447007988 -1.420463615551582]
 [1.0 0.0 0.0 1.1475343068237058 1.232653363453549]
 [0.0 1.0 0.0 1.4379472069688968 1.5749910381638885]
 [1.0 0.0 0.0 -0.7401495441200351 -0.5646194287757332]]


In [72]:
print(x_test)

[[0.0 1.0 0.0 -1.4661817944830124 -0.9069571034860727]
 [1.0 0.0 0.0 -0.44973664397484414 0.2056403393225306]]


**feature scaling which will allow to put all our features on the same scale.**
Why do we need to do this? 
Well, that's because for some of the machinery models that's in order to avoid some features to be dominated by other features in such a way that the dominated features are not even considered by the machinery model.
 Now, you also need to be aware that we won't have to apply feature scaling for all the machinery models, just for some of them. 
The main two feature scaling techniques that indeed put all your features in the same scale. And these two techniques are first, standardization and then you have normalization, 



**Now the question is also much asked by the data science community. Should we go for standardization or normalization?**
Well, we're gonna be here very pragmatic. Normalization is recommended when you have a normal distribution in most of your features. This will be a great feature scaling technique in that case. Standardization actually works well all the time. It will do the job all the time. Therefore, since this is a technique that will work all the time and this is a technique that is more recommended. 


we understood in the previous tutorial that feature scaling must be applied after the split while you understand that **we won't apply feature scaling on the whole matrix of features X but of course on both X-train and X-test separately.** **And actually the scaler will be fitted to only X-train and then we'll transform X-test.** You know, we'll apply feature scaling on X-test because indeed since X-test is something that we're not supposed to have during the training, but only after like when going in production, well, we're not allowed to fit our feature scanning tool on the test set right? By fitting the feature scanning tool on the test set that means that we're gonna get the mean of the whole set and then the standard deviation in the feature. No, we don't have the right to do this because X-test is supposed to be something new and therefore we'll just get the mean of the values in X-train, then get the standard deviation of the values in X-train, then apply this formula to transform all the values in X-train, and then apply that same formula but with the same mean, and send the deviation of the values in X-train to scale the values of X-test.



We're about to use is a class called Standard Scaler in which will exactly perform standardization on both your matrix of features of the training set and the matrix of features of the test set.


**do we have to apply feature scaling you know, standardization to the dummy variables in the matrix of features?**


The answer is no, The answer is no, because simply, well remember the goal of standardizationor feature scaling in general, it is to have all the values of the features in the same range. And since I told you that standardization actually transforms your features so that they take values between more or less minus three and plus three, while since here our dummy variables already take values between minus three and plus three because they're equal to either one or zero.
Well, there is nothing extra to be done here with standardization.

And actually, standardization will only make it worse
because indeed it will still transform these values between minus three and plus three. But then you will totally lose the interpretation of these variables.

In other words, you will lose the information of which country corresponds to the observation. If we apply it on the dummy variables we will get nonsense numerical values and we will be absolutely uncapable to say, which tuple of three values here correspond to which country.


**difference between fit and transform.**

Fit will just get the mean and send the deviation of each of your features, and transform will apply this formula to indeed transform your values so that they can all be in the same scale.


And now I hope you know what the next step is going to be, and you won't fall into the trap. Now we have to also transform our matrix of features of the test set, meaning x_test this matrix of features.

But since this data is like new data,which we get, you know, later on in production, well, for this data I will only apply the transform method because indeed the features of the test set need to be scaled by the same scaler that was used on the training set.

We cannot get a new scaler. You know, if we applied the fit transform method here on x_test, we would get a new scaler. And that would absolutely not make sense,because x_test will actually be the input of the predict function that will return the predictions, you know, after the machine learning model is trained.

And since this machine learning model will be trained with a particular scaler, you know, the scaler applied on the training set, well, in order to make predictions that will be congruent with the way the model was trained, well, we need to apply the same scaler that was used on the training set onto the test set,

so that we can get indeed the same transformation,and therefore in the end, some relevant predictions with the predict method applied to x_test.
So here, it's clearly the transform method that must only be applied,



